# Functions and `DataFrame.apply()` Method 

We have already discussed how to add a new column to a `DataFrame` that is a simple function of existing columns.  

Suppose the situation is a little more complicated, and that the column we want to add is some kind of custom (user defined) function of existing columns.

In this tutorial we discuss two ways of doing this:

1. A `for` loop

2. `DataFrame.apply()`

We will use a finance task to motivate these two techniques: calculating the payoffs of expiring options.

### Defining Functions

Defining functions in Python is straightforward.  They syntax is simply `def function_name(arguments):`.  The following function squares two numbers.

In [1]:
def square(x):
    sq = x ** 2
    return(sq)

Let's verify that our function works:

In [2]:
print(square(2))
print(square(5))

4
25


**Code Challenge:** Write a `cube()` function that cubes a number, and along the way, verify that indentation is required after the `def` statement.

In [3]:
def cube(x):
    cb = x ** 3
    return(cb)

print(cube(3))

27


### Option Payoff Function

Let's now write a more financially interesting function.

Options are insurance contracts that are written on top of an underlying stock, much like car insurance is written *on top* of your car.  There are two types of options: *puts* and *calls*.  Put options protect you from the stock price going too low, while call options protect you from the stock price going too high.  Both types have a feature called a *strike* price, which acts much like the deductable of your car insurance.  Options expire sometime in the future,  and the payoff (payout) of the option at the time of the expiration is as follows:

Let $K$ be the strike price of an option, and let $S_{T}$ price of its underlying at the time of expiration.  Then the payoff of each type of option is as follows:

1. **call**: $\max(S_T - K, 0)$

2. **put**: $\max(K - S_T, 0)$

We can codify this as follows:

In [4]:
def option_payoff(cp, strike, upx):
    if cp == 'call':
        payoff = max(upx - strike, 0)
    elif cp == 'put':
        payoff = max(strike - upx, 0)
    
    return payoff

Let's verify that our function works:

In [5]:
print(option_payoff("call", 100, 110))
print(option_payoff("put", 100, 110))
print(option_payoff("call", 100, 90))
print(option_payoff("put", 100, 90))

10
0
0
10


### Loading Packages

Let's now load the packages that we will need.

In [6]:
import numpy as np
import pandas as pd

### Reading-In Data

Next, let's read in a data file called `spy_expiring_options.csv`. 

This data set consists of 21 different options on `SPY` that expired on November 16, 2018.  

The `upx` column is the settle price of `SPY` from that day, and it will be used to calculate the payoff of each of these options.

In [7]:
df_opt = pd.read_csv("spy_expiring_options.csv")
df_opt = df_opt.round(2)
df_opt.head()

,underlying,upx,type,expiration,data_date,strike
0,SPY,273.73,put,2018-11-16,2018-11-16,270.0
1,SPY,273.73,put,2018-11-16,2018-11-16,270.5
2,SPY,273.73,put,2018-11-16,2018-11-16,271.0
3,SPY,273.73,put,2018-11-16,2018-11-16,271.5
4,SPY,273.73,put,2018-11-16,2018-11-16,272.0


### Initializing Payoff Columns

Our ultimate objective is to add a column of option payoffs to `df_opt`.  We are going to accomplish this task using two different methods: (1) a `for` loop; (2) the `DataFrame.apply()` method.  

As a first step, let's add two columns to `df_opt`,  one for each method, and then initialize them both with `np.nan`, which is a special data type that represents missing numerical data.

In [8]:
df_opt['payoff_loop'] = np.nan
df_opt['payoff_apply'] = np.nan
df_opt.head()

,underlying,upx,type,expiration,data_date,strike,payoff_loop,payoff_apply
0,SPY,273.73,put,2018-11-16,2018-11-16,270.0,NaN,NaN
1,SPY,273.73,put,2018-11-16,2018-11-16,270.5,NaN,NaN
2,SPY,273.73,put,2018-11-16,2018-11-16,271.0,NaN,NaN
3,SPY,273.73,put,2018-11-16,2018-11-16,271.5,NaN,NaN
4,SPY,273.73,put,2018-11-16,2018-11-16,272.0,NaN,NaN


### Calculate `option_payoff` via `for` loop

Let's iterate through `df_opt` with a `for` loop and calculate the payoffs one by one.  Notice that we are useing the `.at` indexer which is specifically designed to grab a single value from a column.

In [9]:
for ix in df_opt.index:
    
    # grabbing data from dataframe
    opt_type = df_opt.at[ix, 'type']
    strike = df_opt.at[ix, 'strike']
    upx = df_opt.at[ix, 'upx']
    
    # calculating payoff
    payoff = option_payoff(opt_type, strike, upx)
    
    # writing payoff to dataframe
    df_opt.at[ix, 'payoff_loop'] = payoff
      
df_opt

,underlying,upx,type,expiration,data_date,strike,payoff_loop,payoff_apply
0,SPY,273.73,put,2018-11-16,2018-11-16,270.0,0.00,NaN
1,SPY,273.73,put,2018-11-16,2018-11-16,270.5,0.00,NaN
2,SPY,273.73,put,2018-11-16,2018-11-16,271.0,0.00,NaN
3,SPY,273.73,put,2018-11-16,2018-11-16,271.5,0.00,NaN
4,SPY,273.73,put,2018-11-16,2018-11-16,272.0,0.00,NaN
5,SPY,273.73,put,2018-11-16,2018-11-16,272.5,0.00,NaN
6,SPY,273.73,put,2018-11-16,2018-11-16,273.0,0.00,NaN
7,SPY,273.73,put,2018-11-16,2018-11-16,273.5,0.00,NaN
8,SPY,273.73,put,2018-11-16,2018-11-16,274.0,0.27,NaN
9,SPY,273.73,put,2018-11-16,2018-11-16,274.5,0.77,NaN


### Calculate `opt_pay` via `.apply()`

The `DataFrame.apply()` method allows us to perform these calculations without explicitly iterating through `df_opt` with a `for` loop.  It is a way to *vectorize* user defined functions.

In order to make use of `.apply()`, we will have to construct our custom payoff function slightly differently.  The following `opt_pay()` function expects as its argument the row of a `DataFrame`:

In [10]:
def opt_pay(row):
    # reading function inputs from DataFrame row
    cp = row['type']
    strike = row['strike']
    upx = row['upx']
    
    # option payoff logic
    if cp == 'call':
        payoff = max(upx - strike, 0)
    elif cp == 'put':
        payoff = max(strike - upx, 0)
    
    return payoff

We can use `.apply()` to calculate the payoffs in a single line of code.

In [11]:
df_opt['payoff_apply'] = df_opt.apply(opt_pay, axis = 1)
df_opt

,underlying,upx,type,expiration,data_date,strike,payoff_loop,payoff_apply
0,SPY,273.73,put,2018-11-16,2018-11-16,270.0,0.00,0.00
1,SPY,273.73,put,2018-11-16,2018-11-16,270.5,0.00,0.00
2,SPY,273.73,put,2018-11-16,2018-11-16,271.0,0.00,0.00
3,SPY,273.73,put,2018-11-16,2018-11-16,271.5,0.00,0.00
4,SPY,273.73,put,2018-11-16,2018-11-16,272.0,0.00,0.00
5,SPY,273.73,put,2018-11-16,2018-11-16,272.5,0.00,0.00
6,SPY,273.73,put,2018-11-16,2018-11-16,273.0,0.00,0.00
7,SPY,273.73,put,2018-11-16,2018-11-16,273.5,0.00,0.00
8,SPY,273.73,put,2018-11-16,2018-11-16,274.0,0.27,0.27
9,SPY,273.73,put,2018-11-16,2018-11-16,274.5,0.77,0.77


**Code Challenge:** Add a column to `df_opt` that identifies if the `upx` is bigger or smaller than `strike`.  Do this by writing a custom function and then using `DataFrame.apply()`.

In [12]:
def big_small(row):
    upx = row['upx']
    strike = row['strike']
    
    if upx >= strike:
        return('bigger')
    else:
        return('smaller')

In [13]:
df_opt['big_small'] = df_opt.apply(big_small, axis = 1)
df_opt

,underlying,upx,type,expiration,data_date,strike,payoff_loop,payoff_apply,big_small
0,SPY,273.73,put,2018-11-16,2018-11-16,270.0,0.00,0.00,bigger
1,SPY,273.73,put,2018-11-16,2018-11-16,270.5,0.00,0.00,bigger
2,SPY,273.73,put,2018-11-16,2018-11-16,271.0,0.00,0.00,bigger
3,SPY,273.73,put,2018-11-16,2018-11-16,271.5,0.00,0.00,bigger
4,SPY,273.73,put,2018-11-16,2018-11-16,272.0,0.00,0.00,bigger
5,SPY,273.73,put,2018-11-16,2018-11-16,272.5,0.00,0.00,bigger
6,SPY,273.73,put,2018-11-16,2018-11-16,273.0,0.00,0.00,bigger
7,SPY,273.73,put,2018-11-16,2018-11-16,273.5,0.00,0.00,bigger
8,SPY,273.73,put,2018-11-16,2018-11-16,274.0,0.27,0.27,smaller
9,SPY,273.73,put,2018-11-16,2018-11-16,274.5,0.77,0.77,smaller


### Related Reading

*WTP* - 8 - Control Flow

*WTP* - 9 - Defining Functions